In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
df_52 = pd.read_csv('score_52.csv')

df_52.head()

,id,mae,mse,crps,log_score,interval_score,state,year,model_id
0,224,242.560562,9.841579e+04,208.193260,-13.403591,3079.842430,CE,2023,21
1,225,315.338436,1.697442e+05,273.255272,-16.166292,4911.727100,CE,2024,21
2,226,2271.685398,1.442219e+07,2195.817933,-74.876275,41274.155335,PR,2023,21
3,227,11040.313332,2.186483e+08,10834.670721,-78.179553,209246.283207,PR,2024,21
4,228,2148.658965,9.386384e+06,2010.943032,-60.561087,35528.054813,MG,2023,21


In [3]:
df_52.shape

(60, 9)

In [4]:
df_26 = pd.read_csv('score_26.csv')

df_26.head()

,id,mae,mse,crps,log_score,interval_score,state,year,model_id
0,224,106.734504,2.011719e+04,87.856623,-12.091311,1119.789090,CE,2023,21
1,225,234.344647,8.958741e+04,201.407722,-15.726707,3468.361083,CE,2024,21
2,226,1484.546513,7.119819e+06,1398.042209,-49.752549,24729.477781,PR,2023,21
3,227,9073.123949,1.749322e+08,8902.667970,-74.556205,171810.702828,PR,2024,21
4,228,1871.883818,7.635668e+06,1697.207088,-40.322018,28098.829927,MG,2023,21


In [5]:
state = 'CE'
year = 2024

df_52_ = df_52.loc[(df_52.state == state) & (df_52.year == year)]
df_26_ = df_26.loc[(df_26.state == state) & (df_26.year == year)]

df_52_.head()

,id,mae,mse,crps,log_score,interval_score,state,year,model_id
1,225,315.338436,169744.198478,273.255272,-16.166292,4911.727100,CE,2024,21
11,133,484.942857,370115.657143,309.246322,-7.484065,2946.551429,CE,2024,22
21,179,285.342857,198200.257143,258.400320,-21.889823,3995.210000,CE,2024,25
30,192,301.236302,162246.668528,297.765920,-98.323295,5764.125207,CE,2024,27
46,202,302.921345,164248.715020,299.469475,-98.348684,5785.684842,CE,2024,28


In [6]:
df_52_.shape

(6, 9)

Cálculo do ranking final: 

In [7]:
def compute_the_rank_by_scores(df_52_, df_26_): 

    S1 = df_52_[['crps', 'model_id']].copy()
    S1['rank'] = df_52_['crps'].rank(method='dense', ascending=True)
    S1['score'] = 'S1'

    S2 = df_26_[['crps', 'model_id']].copy()
    S2['rank'] = df_26_['crps'].rank(method='dense', ascending=True)
    S2['score'] = 'S2'

    S3 = df_52_[['log_score', 'model_id']].copy()
    S3['rank'] = df_52_['log_score'].rank(method='dense', ascending=False)
    S3['score'] = 'S3'

    S4 = df_26_[['log_score', 'model_id']].copy()
    S4['rank'] = df_26_['log_score'].rank(method='dense', ascending=False)
    S4['score'] = 'S4'

    S5 = df_52_[['interval_score', 'model_id']].copy()
    S5['rank'] = df_52_['interval_score'].rank(method='dense', ascending=True)
    S5['score'] = 'S5'

    S6 = df_26_[['interval_score', 'model_id']].copy()
    S6['rank'] = df_26_['interval_score'].rank(method='dense', ascending=True)
    S6['score'] = 'S6'

    S = pd.concat([S1, S2, S3, S4, S5, S6]).reset_index(drop = True)

    S = pd.pivot_table(S, values = 'rank', index='model_id',
                      columns = ['score'])

    S.columns = 'rank_' + S.columns 

    return S

def compute_the_final_rank(S):
    v_r_end = []
    for model_id in S.index: 
    
        rank_final = ((1/S.loc[model_id]['rank_S1']) +
                     (1/S.loc[model_id]['rank_S2']) +
                     (1/S.loc[model_id]['rank_S3']) + 
                     (1/S.loc[model_id]['rank_S4']) + 
                     (1/S.loc[model_id]['rank_S5']) + 
                     (1/S.loc[model_id]['rank_S6']))
    
        v_r_end.append(rank_final)
    
    df_rank = pd.DataFrame()
    
    df_rank['model_id'] = S.index
    df_rank['rank_values'] = v_r_end

    df_rank['rank'] = df_rank['rank_values'].rank(method='dense', ascending=False) 
    df_rank['state'] = state
    df_rank['year'] = year

    df_rank = df_rank.set_index('model_id')

    df_rank = df_rank.rename(columns = {'rank':'rank_final'})

    df_rank = S.merge(df_rank[['rank_final', 'state', 'year']], left_index = True, right_index = True)

    for c in ['rank_S1', 'rank_S2', 'rank_S3', 
                                         'rank_S4', 'rank_S5', 'rank_S6', 'rank_final']:

        df_rank[c] = df_rank[c].astype(int)

    return df_rank.sort_values(by ='rank_final')


In [8]:
from itertools import product 

In [9]:
%%time 
for state, year in product(['AM','CE','GO','PR','MG'], [2023, 2024]):
    df_52_ = df_52.loc[(df_52.state == state) & (df_52.year == year)]
    df_26_ = df_26.loc[(df_26.state == state) & (df_26.year == year)]

    S_end = compute_the_rank_by_scores(df_52_, df_26_)

    R_end = compute_the_final_rank(S_end)
    R_end.to_csv(f'scores/ranking_{state}_{year}.csv')
    

CPU times: user 62.4 ms, sys: 3.33 ms, total: 65.7 ms
Wall time: 66.2 ms


In [10]:
R_end

,rank_S1,rank_S2,rank_S3,rank_S4,rank_S5,rank_S6,rank_final,state,year
model_id,,,,,,,,,
25,4,2,1,1,1,1,1,MG,2024
21,1,1,4,4,3,3,2,MG,2024
22,2,3,2,2,2,2,3,MG,2024
30,6,4,3,3,4,4,4,MG,2024
28,3,5,5,5,5,5,5,MG,2024
27,5,6,6,5,6,6,6,MG,2024


In [11]:
import seaborn as sns
cm = sns.color_palette("light:b_r", as_cmap=True)

R_end.style.background_gradient(subset= ['rank_S1', 'rank_S2', 'rank_S3', 
                                         'rank_S4', 'rank_S5', 'rank_S6', 'rank_final'], cmap=cm)

,rank_S1,rank_S2,rank_S3,rank_S4,rank_S5,rank_S6,rank_final,state,year
model_id,,,,,,,,,
25,4,2,1,1,1,1,1,MG,2024
21,1,1,4,4,3,3,2,MG,2024
22,2,3,2,2,2,2,3,MG,2024
30,6,4,3,3,4,4,4,MG,2024
28,3,5,5,5,5,5,5,MG,2024
27,5,6,6,5,6,6,6,MG,2024


Final rank for state:

In [12]:
state = 'AM'

In [13]:
df_r_2023 = pd.read_csv(f'scores/ranking_{state}_2024.csv',index_col = 'model_id')

df_r_2023.head(6)

,rank_S1,rank_S2,rank_S3,rank_S4,rank_S5,rank_S6,rank_final,state,year
model_id,,,,,,,,,
22,1,1,1,1,3,3,1,AM,2024
25,2,2,2,2,1,1,2,AM,2024
30,3,3,3,3,2,2,3,AM,2024
21,4,4,4,4,4,4,4,AM,2024
28,5,5,5,5,5,5,5,AM,2024
27,6,6,6,5,6,6,6,AM,2024


In [14]:
for state in ['AM', 'CE', 'GO', 'PR', 'MG']:
    df_r_2023 = pd.read_csv(f'scores/ranking_{state}_2023.csv', index_col = 'model_id')
    
    df_r_2024 = pd.read_csv(f'scores/ranking_{state}_2024.csv', index_col = 'model_id')
    
    v_r_end = []
    for model_id in df_r_2023.index: 
        
            rank_final = ((1/df_r_2023.loc[model_id]['rank_final']) +
                         (1/df_r_2024.loc[model_id]['rank_final']))
        
            v_r_end.append(rank_final)
        
    df_rank = pd.DataFrame()
        
    df_rank['model_id'] = df_r_2023.index
    df_rank['rank_values'] = v_r_end
    
    df_rank['rank'] = df_rank['rank_values'].rank(method='dense', ascending=False) 
    df_rank['state'] = state
   
    df_rank.to_csv(f'scores/ranking_{state}.csv', index = False)

In [15]:
df_rank.sort_values('rank_values', ascending = False)

,model_id,rank_values,rank,state
3,25,1.333333,1.0,MG
0,30,1.250000,2.0,MG
1,21,1.000000,3.0,MG
2,22,0.833333,4.0,MG
4,28,0.450000,5.0,MG
5,27,0.366667,6.0,MG


In [16]:
df_r_2023

,rank_S1,rank_S2,rank_S3,rank_S4,rank_S5,rank_S6,rank_final,state,year
model_id,,,,,,,,,
30,3,3,1,2,1,1,1,MG,2023
21,1,1,4,4,4,4,2,MG,2023
22,4,4,2,1,2,2,2,MG,2023
25,2,2,3,3,3,3,3,MG,2023
28,5,5,5,5,5,5,4,MG,2023
27,6,6,5,5,6,6,5,MG,2023


In [17]:
df_r_2024

,rank_S1,rank_S2,rank_S3,rank_S4,rank_S5,rank_S6,rank_final,state,year
model_id,,,,,,,,,
25,4,2,1,1,1,1,1,MG,2024
21,1,1,4,4,3,3,2,MG,2024
22,2,3,2,2,2,2,3,MG,2024
30,6,4,3,3,4,4,4,MG,2024
28,3,5,5,5,5,5,5,MG,2024
27,5,6,6,5,6,6,6,MG,2024


In [18]:
df_rank_ = pd.read_csv(f'scores/ranking_AM.csv', index_col = 'model_id')

df_rank_

,rank_values,rank,state
model_id,,,
30,1.333333,1.0,AM
21,0.750000,2.0,AM
22,1.333333,1.0,AM
25,0.750000,2.0,AM
27,0.366667,3.0,AM
28,0.366667,3.0,AM


Final global rank:

In [19]:
list_of_dfs = [] 

for state in ['AM', 'CE', 'GO', 'PR', 'MG']: 

    df_rank_ = pd.read_csv(f'scores/ranking_{state}.csv', index_col = 'model_id', usecols = ['model_id', 'rank'])
    df_rank_ = df_rank_.rename(columns = {'rank': f'rank_{state}'} )

    list_of_dfs.append(df_rank_)
    
df_rank_states = pd.concat(list_of_dfs,axis =1)

df_rank_states    

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG
model_id,,,,,
30,1.0,1.0,2.0,1.0,2.0
21,2.0,2.0,3.0,2.0,3.0
22,1.0,4.0,1.0,1.0,4.0
25,2.0,3.0,4.0,2.0,1.0
27,3.0,5.0,6.0,4.0,6.0
28,3.0,5.0,5.0,3.0,5.0


In [23]:
v_r_end = []

for model_id in df_rank_states.index: 

    rank_final = (
                         (1/df_rank_states.loc[model_id]['rank_AM']) +
                         (1/df_rank_states.loc[model_id]['rank_CE']) + 
                         (1/df_rank_states.loc[model_id]['rank_GO']) + 
                         (1/df_rank_states.loc[model_id]['rank_MG']) + 
                         (1/df_rank_states.loc[model_id]['rank_PR']))
    v_r_end.append(rank_final)
    
df_rank = pd.DataFrame()
    
df_rank['model_id'] = df_rank_states.index
df_rank['rank_values'] = v_r_end
    
df_rank['rank'] = df_rank['rank_values'].rank(method='dense', ascending=False) 

df_rank.set_index('model_id',inplace = True)

df_rank = df_rank.rename(columns = {'rank': 'rank_global'})

df_rank = df_rank_states.merge(df_rank['rank_global'], left_index = True, right_index = True)

df_rank = df_rank.sort_values(by = 'rank_global')

df_rank = df_rank.astype(int)

df_rank.head(6)

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,rank_global
model_id,,,,,,
30,1,1,2,1,2,1
22,1,4,1,1,4,2
25,2,3,4,2,1,3
21,2,2,3,2,3,4
28,3,5,5,3,5,5
27,3,5,6,4,6,6


In [24]:
df_rank.style.background_gradient(cmap=cm)

,rank_AM,rank_CE,rank_GO,rank_PR,rank_MG,rank_global
model_id,,,,,,
30,1,1,2,1,2,1
22,1,4,1,1,4,2
25,2,3,4,2,1,3
21,2,2,3,2,3,4
28,3,5,5,3,5,5
27,3,5,6,4,6,6
